In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from tpot.config import classifier_config_dict

import pandas as pd
import numpy as np
import os
import glob

In [2]:
accuracy_ls = []
n_gen = 5
n_pop = 5
personal_config = classifier_config_dict

dat_name = 'simulatedGenex'
tpot_data = pd.read_csv('simulatedGenex.csv')
Xdata = tpot_data.loc[:, tpot_data.columns != 'class']
Xdata = Xdata.drop(Xdata.columns[0], axis=1)
Ydata = tpot_data['class']

subset_df = pd.read_csv('subsets.csv')
all_features = ";".join(subset_df['Features'].tolist())
uniq_features = set(all_features.split(';')) # unique features in all subsets
overlap_features = list(uniq_features.intersection(set(list(Xdata.columns.values))))
X_subset = Xdata[overlap_features]

X_train, X_test, y_train, y_test = train_test_split(X_subset, Ydata, random_state = 1618,
                                                    train_size=0.75, test_size=0.25)

del X_subset
del Xdata
del Ydata
del tpot_data

seed = 1618
tpot = TPOTClassifier(generations=n_gen, config_dict=personal_config,
                      population_size=n_pop, verbosity=2, random_state=seed,
                      early_stop=10,
                      template='XGBClassifier')

In [3]:
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.5805190952910642
Generation 2 - Current best internal CV score: 0.5805190952910642
Generation 3 - Current best internal CV score: 0.5805190952910642
Generation 4 - Current best internal CV score: 0.5805190952910642
Generation 5 - Current best internal CV score: 0.5805190952910642

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.5, max_depth=4, min_child_weight=5, n_estimators=100, nthread=1, subsample=0.8)
0.52


FileNotFoundError: [Errno 2] No such file or directory: 'pipelines/XGBoostsimulatedGenex1618.py'

In [7]:
tpot.export('XGBoost' + dat_name + str(seed) + '.py')
accuracy_ls.append([tpot._optimized_pipeline_score, tpot.score(X_test, y_test)])
accuracy_mat = pd.DataFrame(accuracy_ls, columns = ['Training CV Accuracy', 'Testing Accuracy'])
accuracy_mat.to_csv("XGBoost" + str(n_gen) + '_' + str(n_pop) + '_' + str(seed) + ".tsv", sep = "\t")
accuracy_mat

,Training CV Accuracy,Testing Accuracy
0,0.580519,0.52
1,0.580519,0.52
